In [ ]:
!git clone https://github.com/YasasviPeruvemba/deblur-gan.git

In [ ]:
# MOUNT DRIVE (WITH DATASET) BEFORE PROCEEDING
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
mkdir /content/deblur-gan/gopro/train -p
mkdir /content/deblur-gan/logs
cp "/content/drive/My Drive/DeblurGANImages/blurred_sharp.zip" /content/deblur-gan/gopro/train/
cd /content/deblur-gan/gopro/train
unzip blurred_sharp.zip
mv "./blurred_sharp/blurred" "./blurred_sharp/blur"
rm -r /content/deblur-gan/gopro/train/__MACOSX
rm blurred_sharp.zip
cd /content/deblur-gan

In [ ]:
%cd /content/deblur-gan/
%pip install -r requirements/requirements.txt
%pip install -e .

In [ ]:
!python scripts/organize_gopro_dataset.py --dir_in=./gopro --dir_out=./images

In [ ]:
# CHECK IF THE PREPROCESSING WORKED
import matplotlib.pyplot as plt

a = plt.imread("./images/train/A/blurred_sharp_257.png")
plt.imshow(a)

In [ ]:
!python scripts/train.py --n_images=8 --batch_size=8 --log_dir ./logs --epoch_num=10

-0.4961452782154083 2044.5028
